In [ ]:
import tensor_comprehensions as tc
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt

import os
import sys
import logging
import time

In [ ]:
# as recommended by the authors for better performance
tune_settings = {
    "threads": 32,
    "generations": 8,
    "pop_size": 25,
    "number_elites": 3
}

In [ ]:
mobilenet_lang = tc.database['small_mobilenet']['lang']

In [ ]:
# tune mobilenet for the dimensions typical of mobilenet

I = torch.randn(128, 56, 56).cuda()
W1 = torch.randn(128, 3, 3).cuda()
B1 = torch.randn(128).cuda()
W2 = torch.randn(128, 128).cuda()
B2 = torch.randn(128).cuda()

small_mobilenet = tc.define(mobilenet_lang, name='small_mobilenet')
small_mobilenet.autotune(I, W1, B1, W2, B2,
                         cache='cache/small_mobilenet_128_56_3_128',
                         **tune_settings,
                         options=tc.Options('conv'))

In [ ]:
# run twice to prep cuda

small_mobilenet(I, W1, B1, W2, B2, cache='cache/small_mobilenet_128_56_3_128.tc')
torch.cuda.synchronize()
small_mobilenet(I, W1, B1, W2, B2, cache='cache/small_mobilenet_128_56_3_128.tc')
torch.cuda.synchronize()

In [ ]:
# test performance using TC

output = torch.zeros(128, 56, 56).cuda()

timings = np.zeros(1000)

for i in range(1000):
    start = time.perf_counter()
    tbmm(I, W1, B1, W2, B2,
         cache='cache/small_mobilenet_128_56_3_128.tc', 
         outputs=output)
    torch.cuda.synchronize()
    timings[i] = time.perf_counter() - start

total_elapsed = np.sum(timings)
print('total time: ' + str(total_elapsed))

# get percentile statistics
percentiles = np.percentile(timings, [0, 50, 90])
print(percentiles)

In [ ]:
# test the torch equivalent

class MobileNet(nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(128, 128, 3, padding=1, groups=128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 1, padding=0),
            nn.ReLU(inplace=True))
    
    def forward(self, x):
        return self.layer(x)

MN = MobileNet()

var_input = Variable(torch.randn(1, 128, 56, 56))
# run a couple times to work out start hiccups
MN(var_input)
torch.cuda.synchronize()
MN(var_input)
torch.cuda.synchronize()


# test performance
timings = np.zeros(1000)
output = torch.randn(1, 128, 56, 56).cuda()

for i in range(1000):
    start = time.perf_counter()
    output = MN(var_input)
    torch.cuda.synchronize()
    timings[i] = time.perf_counter() - start

total_elapsed = np.sum(timings)
print('total time: ' + str(total_elapsed))

# get percentile statistics
percentiles = np.percentile(timings, [0, 50, 90])
print(percentiles)